In [1]:
import quartz
import torch
import dgl
from icecream import ic

/home/jinjun/miniconda3/envs/quartz/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gate_set = ['h', 'cx', 't', 'tdg']
ecc_file = 'bfs_verified_simplified.json'
no_increase = False
include_nop = True
quartz_context = quartz.QuartzContext(
    gate_set=gate_set,
    filename=ecc_file,
    # TODO  we need to include xfers that lead to gate increase when training?
    # we may exclude them when generating the dataset for pre-training
    # TODO  to make the task easier, we exclude those xfers currently
    no_increase=no_increase,
    include_nop=include_nop,
)
num_xfers = quartz_context.num_xfers
parser = quartz.PyQASMParser(context=quartz_context)
ic(num_xfers)


ic| num_xfers: 3904


3904

In [3]:
qasm_str = 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\nh q[4];\ncx q[3],q[4];\ntdg q[4];\ncx q[2],q[4];\nt q[4];\ncx q[3],q[4];\ntdg q[4];\ncx q[3],q[2];\ntdg q[2];\ncx q[3],q[2];\nt q[3];\ncx q[2],q[4];\nh q[3];\nt q[4];\ncx q[1],q[3];\ntdg q[3];\ncx q[0],q[3];\nt q[3];\ncx q[1],q[3];\ntdg q[3];\ncx q[0],q[3];\ncx q[0],q[1];\nt q[3];\ntdg q[1];\nh q[3];\ncx q[0],q[1];\ncx q[3],q[4];\nt q[0];\nt q[1];\nt q[4];\ncx q[2],q[4];\ntdg q[4];\ncx q[3],q[4];\nt q[4];\ncx q[2],q[4];\ntdg q[4];\ncx q[2],q[3];\nh q[4];\nt q[3];\ncx q[2],q[3];\ntdg q[3];\nh q[3];\ncx q[1],q[3];\nt q[3];\ncx q[0],q[3];\ntdg q[3];\ncx q[1],q[3];\nt q[3];\ncx q[0],q[3];\ncx q[0],q[1];\ntdg q[3];\nt q[1];\nh q[3];\ncx q[0],q[1];\ntdg q[0];\ntdg q[1];\n'
dag = parser.load_qasm_str(qasm_str)
graph = quartz.PyGraph(context=quartz_context, dag=dag)

In [5]:
dgl_g = graph.to_dgl_graph()

In [10]:
int(torch.multinomial(torch.Tensor([1,2,3,4]), 1))

3

In [7]:
next_graph, next_nodes = \
    graph.apply_xfer_with_local_state_tracking(
        xfer=quartz_context.get_xfer_from_id(id=action.xfer),
        node=graph.get_node_from_id(id=action.node)
    )

int

In [8]:
avxfers = []
for i in range(graph.num_nodes):
    avxfers.append(
        graph.available_xfers(
            context=quartz_context,
            node=graph.get_node_from_id(id=i)
        )
    )
avxfers

[[3569, 3667, 3903],
 [3649, 3775, 3903],
 [3633, 3657, 3697, 3903],
 [1771, 1772, 1951, 3649, 3775, 3903],
 [1421, 3675, 3903],
 [438,
  482,
  483,
  484,
  485,
  486,
  487,
  1403,
  1404,
  1405,
  1406,
  1407,
  1408,
  1463,
  1665,
  1666,
  1667,
  1668,
  1669,
  1670,
  3649,
  3775,
  3903],
 [3633, 3657, 3697, 3903],
 [1339, 1340, 1984, 2244, 3599, 3600, 3649, 3775, 3903],
 [1399, 1517, 1537, 2247, 3217, 3657, 3903],
 [1016,
  1159,
  1581,
  1658,
  1831,
  1853,
  1895,
  1896,
  2655,
  3009,
  3534,
  3649,
  3672,
  3775,
  3903],
 [3551, 3903],
 [1951, 3649, 3775, 3903],
 [3569, 3667, 3903],
 [3675, 3899, 3903],
 [3649, 3775, 3903],
 [3633, 3657, 3697, 3903],
 [1951, 3649, 3775, 3903],
 [3675, 3903],
 [2151, 3649, 3775, 3903],
 [400, 1539, 2356, 3633, 3657, 3697, 3903],
 [438,
  482,
  483,
  484,
  485,
  486,
  487,
  1245,
  1336,
  1403,
  1404,
  1405,
  1406,
  1407,
  1408,
  1665,
  1666,
  1667,
  1668,
  1669,
  1670,
  3649,
  3775,
  3903],
 [3599, 3600

In [19]:
qasm2 = graph.to_qasm_str()
g2 = quartz.PyGraph(context=quartz_context, dag=parser.load_qasm_str(qasm2))
avxfers2 = []
for i in range(g2.num_nodes):
    avxfers2.append(
        g2.available_xfers(
            context=quartz_context,
            node=g2.get_node_from_id(id=i)
        )
    )
avxfers2 == avxfers

True

In [30]:
max([])

ValueError: max() arg is an empty sequence